# List Action Analytics

In order to understand what helps us collect debt, we look at the positive results of collection calls: PTP. We try to understand what influences PTP. Surely customers who have good credit is more likely to make a promise, but can one remove those obvious influence from the call results and focus on the resources we can control such as call time and call agents?

In [1]:
# This chunk of code make it possible to use src functions
import sys
import os
module_path = os.path.abspath(os.path.join('../src'))

if module_path not in sys.path:
    sys.path.append(module_path)
    
from dsproject import dsproject

dsp = dsproject()

In [2]:
dsp.data_directory = os.path.join(os.path.dirname(dsp.data_directory), '201510_201610')

In [3]:
df = dsp.read_data('ListAction')

In [4]:
from preprocess import *

df['date'] = df['FDTACTIONDATE'].apply(lambda d: d.split(" ")[0])
df['time'] = df['FDTACTIONDATE'].apply(lambda d: d.split(" ")[1])

In [5]:
df['date'] = parse_dates(df['date'], '%d/%m/%Y')

In [2]:
db_soc = dsp.read_data('DB_SOC')

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
len(db_soc.index)

1209383

### Merging dbsoc with list action

In [7]:
db_soc['FileDate'] = parse_dates(db_soc['FileDate'], format='%Y%m%d')

In [8]:
db_soc = transform_range_day(db_soc)

In [9]:
#Generate unique filedate table
unique_filedate = db_soc[['BILLCycle', 'FileDate']].drop_duplicates()

unique_filedate['Next_FileDate'] = unique_filedate.groupby('BILLCycle')[['BILLCycle','FileDate']].shift(-1)
unique_filedate['Previous_FileDate'] = unique_filedate.groupby('BILLCycle')[['BILLCycle','FileDate']].shift(1)
db_soc = db_soc.merge(unique_filedate, on=['BILLCycle', 'FileDate'])

unique_filedate = db_soc[['HASH_AR_ID', 'FileDate', 'Previous_FileDate', 'Next_FileDate']].drop_duplicates()

In [10]:
prep = df.merge(unique_filedate, on='HASH_AR_ID')
prep = prep.loc[(prep['date'] >= prep['FileDate']) & (prep['date'] < prep['Next_FileDate'])]

In [11]:
prep.head()

,FSZCOLLECTORCODE,FSZCOLLECTORSUP,FSZACTIONCODE,FDTACTIONDATE,FResultcategory,FSZRESULTCODE,FDTRESULTDATE,FISRNO,ACTION_DESC,FDTNXTACTDATE,...,NEXTACTION_DESC,ACTIONCOLL,StampDate,HASH_LPM_CST_ID,HASH_AR_ID,date,time,FileDate,Previous_FileDate,Next_FileDate
12,NaN,UPM01SP,DC,15/10/2015 14:47:28,PTP,PTP,20151015,1148803632,Call (Dialer),NaN,...,NaN,UPM0109,20151015,-6799669851245407158,-4749283152832854685,2015-10-15,14:47:28,2015-10-15,NaT,2015-11-12
18,NaN,UPM01SP,IB,22/10/2015 9:00:28,PTP,PTP,20151022,1157659369,ÅÙ¡¤éÒµÔ´µèÍà¢éÒ Inbound,NaN,...,NaN,INB0209,20151022,-6799669851245407158,-4749283152832854685,2015-10-22,9:00:28,2015-10-15,NaT,2015-11-12
25,NaN,UPF00SP,DC,19/4/2016 15:52:23,PTP,PTP,20160419,1779810434,Call (Dialer),NaN,...,NaN,UPF0105,20160419,-6799669851245407158,-4749283152832854685,2016-04-19,15:52:23,2016-04-19,2016-03-16,2016-05-13
31,NaN,UPF00SP,DC,27/4/2016 10:01:20,PTP,PTP,20160427,1818612049,Call (Dialer),NaN,...,NaN,UPF0119,20160427,-6799669851245407158,-4749283152832854685,2016-04-27,10:01:20,2016-04-19,2016-03-16,2016-05-13
37,NaN,UPF00SP,DC,9/5/2016 14:36:49,UNS,NOA,20160509,1864598675,Call (Dialer),NaN,...,NaN,UPF0101,20160509,-6799669851245407158,-4749283152832854685,2016-05-09,14:36:49,2016-04-19,2016-03-16,2016-05-13


In [12]:
prep = prep.merge(db_soc, on=['HASH_AR_ID', 'FileDate'])

In [13]:
prep['Days_past'] = (prep['date']-prep['FileDate'])

In [14]:
prep['Days_past'] = prep['Days_past'].apply(lambda d: d.days)

In [15]:
prep['time'] = parse_dates(prep['time'], format='%H:%M:%S')

In [16]:
prep['Hours'] = prep['time'].apply(lambda t: t.hour)

In [17]:
merge = prep

In [18]:
b1 = merge.loc[merge['Range_day'] == 2]
b2 = merge.loc[merge['Range_day'] == 3]
b3 = merge.loc[merge['Range_day'] == 4]

b_list = [b1,b2,b3]

### Making Models

In [ ]:
category_columns = ['Hours','ACTIONCOLL', 'PortFolio', 'ProductName', 'BILLCycle', 'Group', 'Card Type New']
numeric_columns = ['OSAMT', 'OSPRINCIPLE', 'Days_past']

In [ ]:
from sklearn import preprocessing
labenc = preprocessing.LabelEncoder()

cat_code = {}
for column in category_columns:
    labenc = preprocessing.LabelEncoder()
    b1[column] = labenc.fit_transform(b1[column].tolist())
    cat_code[column] = labenc

In [ ]:
y_onehot_b1 = pd.get_dummies(b1['FResultcategory'])
y_onehot_b2 = pd.get_dummies(b2['FResultcategory'])
y_onehot_b3 = pd.get_dummies(b3['FResultcategory'])

y_list = [y_onehot_b1,y_onehot_b2,y_onehot_b3]

data_num = [(b1[numeric_columns],y_onehot_b1),(b2[numeric_columns],y_onehot_b2),(b3[numeric_columns],y_onehot_b3)]
data_nom = [(b1[category_columns],y_onehot_b1),(b2[category_columns],y_onehot_b2),(b3[category_columns],y_onehot_b3)]

In [ ]:
from sklearn.feature_selection import chi2, f_classif
import numpy as np

def test_feature(x, y, func=f_classif):
    f_group1 = func(x, y)
    f_group1_table = pd.DataFrame(np.array([list(x.columns), f_group1[0], f_group1[1]]).T,
                                  columns=['column_name', 'f_value', 'p_value'])
    f_group1_table['f_value'] = f_group1_table['f_value'].astype(np.float64)
    f_group1_table['p_value'] = f_group1_table['p_value'].astype(np.float64)
    col_to_drop = f_group1_table.loc[f_group1_table['p_value'] > 0.001]['column_name'].tolist()
    return f_group1_table, col_to_drop

In [ ]:
df_numeric = []

for d in data_num:
    b,y = d[0], d[1]
    result_table, col_to_drop = test_feature(b, y['PTP'])
    df_numeric1 = b.drop(col_to_drop, axis=1)
    df_numeric.append(df_numeric1)

In [ ]:
df_nominal = []

for d in data_nom:
    b,y = d[0], d[1]
    result_table, col_to_drop = test_feature(b, y['PTP'],chi2)
    df_cat1 = b.drop(col_to_drop, axis=1)
    df_nominal.append(df_cat1)

In [ ]:
df_nominal[0] = pd.get_dummies(df_nominal[0], columns = ['Hours','ACTIONCOLL', 'Group'])
df_nominal[1] = pd.get_dummies(df_nominal[1], columns = ['Hours','ACTIONCOLL', 'Group'])
df_nominal[2] = pd.get_dummies(df_nominal[2], columns = ['Hours','ACTIONCOLL', 'Group'])

In [ ]:
x = []
for i in range(3):
    x.append(pd.concat([df_nominal[i],df_numeric[i]],axis = 1))

In [ ]:
result_table, col_to_drop = test_feature(x[0], y_list[0]['PTP'])

In [ ]:
x[0] = x[0].drop(col_to_drop, axis=1)

In [ ]:
len(x[0].columns)

# Model Training

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report

def test_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)
    yhat = model.predict(x_test)
    result = classification_report(y_test, yhat)
    print(result)
    return model

mod = []

for i in [0]:
    
    print ("Bucket " + (str)(i+1))
    print ("Spliting training/testing set")
    
    x_train, x_test, y_train, y_test = train_test_split(x[i], y_list[i]['PTP'], test_size=0.2)
    
    print ("Building Model")

    estim = {'RFC': RFC(n_estimators=10, n_jobs=-1, oob_score=True, min_samples_leaf=10, random_state=42)}
    mod1 = {}
    for key, value in estim.items():
        
        print ("Testing Model for Bucket " + (str)(i+1) + " using " + key )
        
        mod1[key] = test_model(value, x_train, y_train, x_test, y_test)
        
    mod.append(mod1)

In [ ]:
i=0
summary = {'Features':x[i].columns, 'Importance':mod[i]['RFC'].feature_importances_}
summary = pd.DataFrame(data = summary)
print ("Summary for Bucket "+ (str)(i+1))
print (summary.sort_values('Importance', ascending = 0))

In [ ]:
cat_code['Hours'][7]

In [ ]:
result_table

In [ ]:
cat_code['ACTIONCOLL'].inverse_transform([356, 361, 359, 358, 360, 395])